In [1]:
import os
import math
from termcolor import colored
import numpy as np
import matplotlib
matplotlib.use('Agg')
import pandas as pd
#from data_helpers import load_data
from keras import callbacks
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
#from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from keras import backend as K
from keras.optimizers import Adam

from keras.layers import Embedding
import string
import numpy as np
import pandas as pd
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import concatenate
from keras.layers import Dropout
import random

import regex
from keras.layers.convolutional import Convolution1D, MaxPooling1D, AveragePooling1D


In [3]:
import pandas as pd
 
dataset=pd.read_csv('/content/50_tweets_per_user.csv')

In [4]:
k=0
X_train=[]
y_train=[]
for i in range(0,50):
  for j in range(k,k+45):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=50

In [5]:
k=45
X_test=[]
y_test=[]
for i in range(0,50):
  for j in range(k,k+5):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=50

In [6]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

In [7]:
train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

In [8]:
np.random.shuffle(train)
np.random.shuffle(test)

In [9]:
train=pd.DataFrame(train)
test=pd.DataFrame(test)

In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
  sentence=str(sentence)
  sentence = sentence.lower()
#  sentence=sentence.replace('{html}',"") 
#  cleanr = re.compile('<.*?>#@')
#  cleantext = re.sub(cleanr, '', sentence)
#  rem_url=re.sub(r'http\S+', '',cleantext)
#  rem_num = re.sub('[0-9]+', '', rem_url)
#  tokenizer = RegexpTokenizer(r'\w+')
#  tokens = tokenizer.tokenize(rem_num)  
#  filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
#  stem_words=[stemmer.stem(w) for w in filtered_words]
#  lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  return sentence


train.iloc[:,0]=train.iloc[:,0].map(lambda s:preprocess(s))
test.iloc[:,0]=test.iloc[:,0].map(lambda s:preprocess(s))

In [13]:
X_train=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]

In [14]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

In [15]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

In [16]:
def create_vocab_set2():
    alphabet = (list(string.ascii_lowercase) + list(string.digits) +
                list(string.punctuation) + ['\n'] + [' '] )
    
    print('alphabet', alphabet)
    #x = input(print('enter'))
    alphabets = []
    for i in range(0, len(alphabet)):
        for j in range(0, len(alphabet)):
              alphabets.append(alphabet[i]+alphabet[j])
    #print(alphabets)
    #x = input(print('enter'))


    vocab_size = len(alphabets)
    check = set(alphabets)

    vocab = {}
    reverse_vocab = {}
    for ix, t in enumerate(alphabets):
        vocab[t] = ix
        reverse_vocab[ix] = t

    return vocab, reverse_vocab, vocab_size, check


In [17]:
maxlen = 140
vocab, reverse_vocab, vocab_size, check =create_vocab_set2()

alphabet ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '\n', ' ']


In [18]:
def encode_data2(x, maxlen, vocab, vocab_size, check):              # For character embedding use this function instead of encode_data

    input_data = np.zeros((len(x), maxlen))
    for dix, sent in enumerate(x):
        counter = 0
        chars = list(sent.lower().replace(" ", ""))
        chars2 = []
        for i in range(len(chars)-1):
            chars2.append(chars[i] + chars[i+1])
        for i,c in enumerate(chars2):
            if counter >= maxlen:
                pass
            else:
                if c in check:
                    counter += 1
                    ix = vocab[c]
                    input_data[dix,counter-1] = ix

    return input_data 

In [19]:
X_train = encode_data2(X_train, maxlen, vocab, vocab_size, check)

In [20]:
X_test= encode_data2(X_test, maxlen, vocab, vocab_size, check)

In [21]:

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X_train,y_train, test_size = 0.25, random_state = 0)

In [26]:
def model2(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter,cat_output):                                                  # For Character Embedding use this model instead of above model
    d = 300                                                             #Embedding Size
    Embedding_layer  = Embedding(vocab_size+1, d, input_length=maxlen)
    inputs = Input(shape=(maxlen,), name='input', dtype='float32')
    embed = Embedding_layer(inputs)
    z = Dropout(0.25)(Dense(dense_outputs, activation='relu')(embed))
    conv = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[0], activation='relu',input_shape=(maxlen, d))(z)
    conv = MaxPooling1D(pool_size=3)(conv)

    conv1 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[1], activation='relu')(conv)
    conv1 = MaxPooling1D(pool_size=3)(conv1)

    conv2 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[2], activation='relu')(conv1)
    conv5 = MaxPooling1D(pool_size=3)(conv2)
    conv = Flatten()(conv5)

    #Two dense layers with dropout of .5
    #z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(conv))
    # z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(z))

    pred = Dense(cat_output, activation='softmax', name='output')(conv)

    model = Model(inputs=inputs, outputs=pred)

    #sgd = SGD(lr=0.001, momentum=0.9)
    #model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [27]:
    nb_filter = 500
    dense_outputs = 256
    filter_kernels = [3,4,5]
    cat_output = 2
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    model = model2(filter_kernels, dense_outputs,maxlen, vocab_size, nb_filter, cat_output)
    model.summary()
    

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 140)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 140, 300)          1470300   
_________________________________________________________________
dense_2 (Dense)              (None, 140, 256)          77056     
_________________________________________________________________
dropout_2 (Dropout)          (None, 140, 256)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 138, 500)          384500    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 46, 500)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 43, 500)           100050

In [ ]:
adam1=Adam(lr=1.00, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])



In [ ]:
model.fit(np.array(X_train), np.array(y_train),validation_data=[np.array(X_eval), np.array(y_eval)],batch_size=32,epochs=15, verbose=1, callbacks=[early_stopping])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1470 samples, validate on 490 samples
Epoch 1/15
1470/1470 [==============================] - 31s 21ms/step - loss: 0.3405 - accuracy: 0.8701 - val_loss: 0.1516 - val_accuracy: 0.9510
Epoch 2/15
1470/1470 [==============================] - 30s 21ms/step - loss: 0.1107 - accuracy: 0.9660 - val_loss: 0.1404 - val_accuracy: 0.9612
Epoch 3/15
1470/1470 [==============================] - 30s 21ms/step - loss: 0.0313 - accuracy: 0.9898 - val_loss: 0.1475 - val_accuracy: 0.9571
Epoch 4/15
1470/1470 [==============================] - 30s 21ms/step - loss: 0.0126 - accuracy: 0.9966 - val_loss: 0.2181 - val_accuracy: 0.9510
Epoch 5/15
1470/1470 [==============================] - 30s 21ms/step - loss: 6.5361e-04 - accuracy: 1.0000 - val_loss: 0.2149 - val_accuracy: 0.9592
Epoch 6/15
1470/1470 [==============================] - 30s 21ms/step - loss: 6.4967e-04 - accuracy: 1.0000 - val_loss: 0.2645 - val_accuracy: 0.9531
Epoch 7/15
1470/1470 [==============================] - 30s 21ms/step

In [ ]:
  y_pred = model.predict(np.array(X_test))

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

In [ ]:
y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

In [ ]:
result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)

In [ ]:
predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)


In [ ]:
print(result)
print(predicted)

[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1]


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

cm

array([[20,  0],
       [ 0, 20]])

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(result, predicted)) 
print('Report : ')
print(classification_report(result, predicted)) 

Confusion Matrix :
[[20  0]
 [ 0 20]]
Accuracy Score : 1.0
Report : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40



In [ ]:
k=980
X_train1=[]
y_train1=[]
for i in range(0,2):
  for j in range(k,k+20):
    X_train1.append(dataset.iloc[j,0])
    y_train1.append(dataset.iloc[j,1])
  k+=1000

In [ ]:
from keras import backend as K1

Using TensorFlow backend.


In [ ]:
file=open("heatmap_authorshipatribution2_new.html","w")
for i in range(0,40):
  type_here=[]
  type_here.append(X_train1[i])
  typr_here=pd.DataFrame(type_here)
  typr_here = encode_data2(type_here, maxlen, vocab, vocab_size, check)
  y_pred = model.predict(typr_here)
  Xtst=typr_here
  class_idx = np.argmax(y_pred[0]) #not needed in this case as only two classes
  class_output = model.output[:, class_idx]
  last_conv_layer = model.get_layer("conv1d_1")
  grads = K.gradients(class_output, last_conv_layer.output)[0]
  pooled_grads = K.mean(grads)
  iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
  pooled_grads_value, conv_layer_output_value = iterate([Xtst])
  

  heatmap = np.mean(conv_layer_output_value, axis=-1)
  heatmap = np.maximum(heatmap,0)
  heatmap /= np.max(heatmap)#normalise values in the prediction
  norm_len = 36/last_conv_layer.output_shape[1]
  html = ""
  if y_pred[0][0]>0.5:
    pred = '90078731'
  else:
    pred = '51964081'
  html += "<span><h3>Based on the description, the model believes that text belongs to {} author ".format(pred)
  html += "<small><br>Confidence: {:.0f}%<br><br></small></h3></span>".format(abs(((y_pred[0][0]*100)-50)*2))
  for j,i in enumerate(type_here[0].split()):
    html += "<span style='background-color:rgba({},0,15,{})'>{} </span>".format(heatmap[math.floor(j/norm_len)]*255,heatmap[math.floor(j/norm_len)]-0.3,i)
  file.write(html)
file.close()   

In [ ]:
 html = ""
if y_pred[0]>0.5:
  pred = '90078731'
else:
  pred = '15401533'
html += "<span><h3>Based on the description, the model believes that text belongs to {} author ".format(pred)
html += "<small><br>Confidence: {:.0f}%<br><br></small></h3></span>".format(abs(((y_pred[0][0]*100)-50)*2))
for j,i in enumerate(type_here[0].split()):
  html += "<span style='background-color:rgba({},0,150,{})'>{} </span>".format(heatmap[math.floor(j/norm_len)]*255,heatmap[math.floor(j/norm_len)]-0.3,i)

HTML(html)
